In [57]:
import json
import requests
import numpy as np
from dateutil.relativedelta import relativedelta
from config import lw_key
import pandas as pd
from datetime import date, timedelta, datetime
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pymongo 
from flask_pymongo import PyMongo
from flask import Flask, render_template, redirect, jsonify

In [2]:
url= "https://api.worldweatheronline.com/premium/v1/weather.ashx?"
past_url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx?"
# query_url =f"{url}appid={api_key}&zip="

In [3]:
zipcode = input("Enter 5-digit Zipcode:")

Enter 5-digit Zipcode:21797


In [4]:
# zip_df = nomi.query_postal_code(zipcode)


In [5]:
# pandas apply pgeocode to ZIPCODE1
# md_zips = df['ZIPCODE1'].apply(nomi.query_postal_code)
# md_zips.head()

In [6]:
response = requests.get(f"{url}key={lw_key}&q={zipcode}&num_of_days=8&tp=24&cc=no&mca=no&aqi=yes&format=json").json()
response = response['data']


# Forecast Data

In [7]:
weather_dict ={
    "Dates":[],
    "Cloudcover":[],
    "Humidity":[],
    "PrecipInch":[],
    "Pressure":[],
    "FeelsLike":[],
    "HeatIndex":[],
    "MaxTemp":[],
    "MinTemp":[],
    "SunHours":[],
    "UVIndex":[],
}

weather_dict["Dates"]= ([response["weather"][i]["date"] for i in range(len(response["weather"]))])
weather_dict["Cloudcover"]=([response["weather"][i]["hourly"][0]["cloudcover"] for i in range(len(response["weather"]))])
weather_dict["Humidity"]= ([response["weather"][i]["hourly"][0]["humidity"] for i in range(len(response["weather"]))])
weather_dict["PrecipInch"]= ([response["weather"][i]["hourly"][0]["precipInches"] for i in range(len(response["weather"]))])
weather_dict["Pressure"]= ([response["weather"][i]["hourly"][0]["pressure"] for i in range(len(response["weather"]))])
weather_dict["FeelsLike"]= ([response["weather"][i]["hourly"][0]["FeelsLikeF"] for i in range(len(response["weather"]))])
weather_dict["HeatIndex"]= ([response["weather"][i]["hourly"][0]["HeatIndexF"] for i in range(len(response["weather"]))])
weather_dict["MaxTemp"]= ([response["weather"][i]["maxtempF"] for i in range(len(response["weather"]))])
weather_dict["MinTemp"]=([response["weather"][i]["mintempF"] for i in range(len(response["weather"]))])
weather_dict["SunHours"]=([response["weather"][i]["sunHour"] for i in range(len(response["weather"]))])
weather_dict["UVIndex"]= ([response["weather"][i]["hourly"][0]["uvIndex"] for i in range(len(response["weather"]))])
weather_df= pd.DataFrame.from_dict(weather_dict,orient='index').transpose()
weather_df= weather_df.apply(pd.to_numeric, errors= 'ignore')
weather_df["TempDelta"] = weather_df.MaxTemp - weather_df.MinTemp
weather_df["BarChange"] =weather_df["Pressure"].pct_change()
weather_df["HeatChange"] =weather_df["HeatIndex"].pct_change()
weather_df["HumChange"] =weather_df["Humidity"].pct_change()
weather_df= weather_df.iloc[1:]
weather_df.head(10)


,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange
1,2021-04-18,62,72,0.0,1014,44,46,54,38,9.1,5,16,0.002967,0.022222,-0.013699
2,2021-04-19,66,78,0.0,1015,46,47,54,43,6.6,5,11,0.000986,0.021739,0.083333
3,2021-04-20,32,71,0.0,1017,51,53,62,42,11.6,6,20,0.001970,0.127660,-0.089744
4,2021-04-21,53,74,0.3,1012,40,46,51,36,9.1,1,15,-0.004916,-0.132075,0.042254
5,2021-04-22,27,65,0.0,1020,36,42,51,33,10.4,3,18,0.007905,-0.086957,-0.121622
6,2021-04-23,43,65,0.0,1021,43,48,58,36,11.6,3,22,0.000980,0.142857,0.000000
7,2021-04-24,81,85,0.3,1015,44,48,57,27,6.7,2,30,-0.005877,0.000000,0.307692


In [41]:
forecast_data = json.loads(weather_df.to_json(orient = "records"))
forecast_data

[{'Dates': '2021-04-18',
  'Cloudcover': 62,
  'Humidity': 72,
  'PrecipInch': 0.0,
  'Pressure': 1014,
  'FeelsLike': 44,
  'HeatIndex': 46,
  'MaxTemp': 54,
  'MinTemp': 38,
  'SunHours': 9.1,
  'UVIndex': 5,
  'TempDelta': 16,
  'BarChange': 0.0029673591,
  'HeatChange': 0.0222222222,
  'HumChange': -0.0136986301},
 {'Dates': '2021-04-19',
  'Cloudcover': 66,
  'Humidity': 78,
  'PrecipInch': 0.0,
  'Pressure': 1015,
  'FeelsLike': 46,
  'HeatIndex': 47,
  'MaxTemp': 54,
  'MinTemp': 43,
  'SunHours': 6.6,
  'UVIndex': 5,
  'TempDelta': 11,
  'BarChange': 0.0009861933,
  'HeatChange': 0.0217391304,
  'HumChange': 0.0833333333},
 {'Dates': '2021-04-20',
  'Cloudcover': 32,
  'Humidity': 71,
  'PrecipInch': 0.0,
  'Pressure': 1017,
  'FeelsLike': 51,
  'HeatIndex': 53,
  'MaxTemp': 62,
  'MinTemp': 42,
  'SunHours': 11.6,
  'UVIndex': 6,
  'TempDelta': 20,
  'BarChange': 0.0019704433,
  'HeatChange': 0.1276595745,
  'HumChange': -0.0897435897},
 {'Dates': '2021-04-21',
  'Cloudcover':

AttributeError: 'list' object has no attribute 'to_json'

In [8]:
new_migraine_df = weather_df.drop("Dates", axis =1)
new_migraine_df

,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange
1,62,72,0.0,1014,44,46,54,38,9.1,5,16,0.002967,0.022222,-0.013699
2,66,78,0.0,1015,46,47,54,43,6.6,5,11,0.000986,0.021739,0.083333
3,32,71,0.0,1017,51,53,62,42,11.6,6,20,0.001970,0.127660,-0.089744
4,53,74,0.3,1012,40,46,51,36,9.1,1,15,-0.004916,-0.132075,0.042254
5,27,65,0.0,1020,36,42,51,33,10.4,3,18,0.007905,-0.086957,-0.121622
6,43,65,0.0,1021,43,48,58,36,11.6,3,22,0.000980,0.142857,0.000000
7,81,85,0.3,1015,44,48,57,27,6.7,2,30,-0.005877,0.000000,0.307692


# Historical Data

In [9]:
dates= input("Dates you had a migraine (YYYY-MM-DD):")

Dates you had a migraine (YYYY-MM-DD):2020-10-18,2020-10-23,2020-10-26,2020-10-28,2020-10-29,2020-11-01,2020-11-02,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-25,2020-11-26,2020-11-28,2020-11-29,2020-11-30,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-09,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-23,2020-12-24,2020-12-26,2020-12-29,2020-12-30,2021-01-03,2021-01-04,2021-01-06,2021-01-07,2021-01-09,2021-01-10,2021-01-12 ,2021-01-15,2021-01-17,2021-01-18,2021-01-20,2021-01-23,2021-01-26,2021-01-27,2021-01-31,2021-02-02,2021-02-06,2021-02-08,2021-02-09,2021-02-10,2021-02-11,2021-02-15,2021-02-16,2021-02-20,2021-02-21,2021-02-24,2021-02-27,2021-03-01,2021-03-05,2021-03-08 ,2021-03-12,2021-03-14,2021-03-16,2021-03-18,2021-03-19,2021-03-23,2021-03-27,2021-03-28


In [10]:
dates=dates.split(",")
dates= [i.strip() for i in dates]
dates_df = pd.DataFrame(dates,columns=['Dates'])
dates_df["Migraine"] = 1


In [11]:
history_dict ={
    "Dates":[],
    "Cloudcover":[],
    "Humidity":[],
    "PrecipInch":[],
    "Pressure":[],
    "FeelsLike":[],
    "HeatIndex":[],
    "MaxTemp":[],
    "MinTemp":[],
    "SunHours":[],
    "UVIndex":[],
    }

In [12]:
for i in range(6,0,-1):
    date= (datetime.today()-relativedelta(months=i)).strftime('%Y-%m-%d')
    enddate=(datetime.today()-relativedelta(months=i-1)).strftime('%Y-%m-%d')
    responses = requests.get(f"{past_url}key={lw_key}&q={zipcode}&date={date}&enddate={enddate}&tp=24&mca=no&aqi=yes&format=json").json()
    responses = responses['data']
    history_dict["Dates"].append([responses["weather"][i]["date"] for i in range(len(responses["weather"]))])
    history_dict["Cloudcover"].append([responses["weather"][i]["hourly"][0]["cloudcover"] for i in range(len(responses["weather"]))])
    history_dict["Humidity"].append([responses["weather"][i]["hourly"][0]["humidity"] for i in range(len(responses["weather"]))])
    history_dict["PrecipInch"].append([responses["weather"][i]["hourly"][0]["precipInches"] for i in range(len(responses["weather"]))])
    history_dict["Pressure"].append([responses["weather"][i]["hourly"][0]["pressure"] for i in range(len(responses["weather"]))])
    history_dict["FeelsLike"].append([responses["weather"][i]["hourly"][0]["FeelsLikeF"] for i in range(len(responses["weather"]))])
    history_dict["HeatIndex"].append([responses["weather"][i]["hourly"][0]["HeatIndexF"] for i in range(len(responses["weather"]))])
    history_dict["MaxTemp"].append([responses["weather"][i]["maxtempF"] for i in range(len(responses["weather"]))])
    history_dict["MinTemp"].append([responses["weather"][i]["mintempF"] for i in range(len(responses["weather"]))])
    history_dict["SunHours"].append([responses["weather"][i]["sunHour"] for i in range(len(responses["weather"]))])
    history_dict["UVIndex"].append([responses["weather"][i]["hourly"][0]["uvIndex"] for i in range(len(responses["weather"]))])

In [13]:
history_df= pd.DataFrame.from_dict(history_dict,orient='index').transpose()
history_df= history_df.apply(pd.to_numeric, errors= 'ignore')


In [14]:
history_df= history_df.apply(pd.Series.explode).reset_index(drop=True)
history_df= history_df.apply(pd.to_numeric, errors= 'ignore')
history_df.reset_index(drop=True)
history_df.head()

,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex
0,2020-10-17,1,69,0.0,1026,46,49,57,40,11.0,3
1,2020-10-18,9,63,0.0,1028,49,51,59,44,11.0,4
2,2020-10-19,40,83,0.0,1026,55,56,65,49,11.0,3
3,2020-10-20,45,83,0.0,1023,61,63,72,54,10.1,4
4,2020-10-21,70,89,0.0,1024,67,68,76,62,7.0,5


In [15]:
history_df["TempDelta"]= history_df.MaxTemp - weather_df.MinTemp
history_df["BarChange"]= history_df["Pressure"].pct_change()
history_df["HeatChange"]= history_df["HeatIndex"].pct_change()
history_df["HumChange"]= history_df["Humidity"].pct_change()
history_df= pd.merge(history_df,dates_df, on = 'Dates', how = 'left')
history_df= history_df.fillna(0)
history_df= history_df.iloc[1:]
history_df.head(10)

,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange,Migraine
1,2020-10-18,9,63,0.0,1028,49,51,59,44,11.0,4,21.0,0.001949,0.040816,-0.086957,1.0
2,2020-10-19,40,83,0.0,1026,55,56,65,49,11.0,3,22.0,-0.001946,0.098039,0.317460,0.0
3,2020-10-20,45,83,0.0,1023,61,63,72,54,10.1,4,30.0,-0.002924,0.125000,0.000000,0.0
4,2020-10-21,70,89,0.0,1024,67,68,76,62,7.0,5,40.0,0.000978,0.079365,0.072289,0.0
5,2020-10-22,7,81,0.0,1023,71,71,79,62,10.9,6,46.0,-0.000977,0.044118,-0.089888,0.0
6,2020-10-23,27,85,0.0,1021,66,67,73,61,10.9,4,37.0,-0.001955,-0.056338,0.049383,1.0
7,2020-10-24,45,84,0.2,1017,63,65,71,57,7.0,4,44.0,-0.003918,-0.029851,-0.011765,0.0
8,2020-10-25,94,84,0.5,1021,40,45,44,38,5.4,2,0.0,0.003933,-0.307692,0.000000,0.0
9,2020-10-26,90,90,0.1,1021,52,52,63,34,5.4,3,0.0,0.000000,0.155556,0.071429,1.0
10,2020-10-27,79,82,0.0,1022,56,57,60,55,8.5,3,0.0,0.000979,0.096154,-0.088889,0.0


In [46]:
history_data = json.loads(history_df.to_json(orient = "records"))
history_data

[{'Dates': '2020-10-18',
  'Cloudcover': 9,
  'Humidity': 63,
  'PrecipInch': 0.0,
  'Pressure': 1028,
  'FeelsLike': 49,
  'HeatIndex': 51,
  'MaxTemp': 59,
  'MinTemp': 44,
  'SunHours': 11.0,
  'UVIndex': 4,
  'TempDelta': 21.0,
  'BarChange': 0.0019493177,
  'HeatChange': 0.0408163265,
  'HumChange': -0.0869565217,
  'Migraine': 1.0},
 {'Dates': '2020-10-19',
  'Cloudcover': 40,
  'Humidity': 83,
  'PrecipInch': 0.0,
  'Pressure': 1026,
  'FeelsLike': 55,
  'HeatIndex': 56,
  'MaxTemp': 65,
  'MinTemp': 49,
  'SunHours': 11.0,
  'UVIndex': 3,
  'TempDelta': 22.0,
  'BarChange': -0.0019455253,
  'HeatChange': 0.0980392157,
  'HumChange': 0.3174603175,
  'Migraine': 0.0},
 {'Dates': '2020-10-20',
  'Cloudcover': 45,
  'Humidity': 83,
  'PrecipInch': 0.0,
  'Pressure': 1023,
  'FeelsLike': 61,
  'HeatIndex': 63,
  'MaxTemp': 72,
  'MinTemp': 54,
  'SunHours': 10.1,
  'UVIndex': 4,
  'TempDelta': 30.0,
  'BarChange': -0.0029239766,
  'HeatChange': 0.125,
  'HumChange': 0.0,
  'Migraine

In [59]:
app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:27017/forecast_db"
mongo =PyMongo(app)
collection = mongo.db.history



In [62]:
history_df_test = pd.DataFrame(list(collection.find()))


In [65]:
#Pre-Processing of Data
hist_ml_df= history_df_test.drop(columns =["Dates","_id","index"])
hist_ml_df

,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange,Migraine
0,9,63,0.0,1028,49,51,59,44,11.0,4,15,0.001949,0.040816,-0.086957,1.0
1,40,83,0.0,1026,55,56,65,49,11.0,3,16,-0.001946,0.098039,0.317460,0.0
2,45,83,0.0,1023,61,63,72,54,10.1,4,18,-0.002924,0.125000,0.000000,0.0
3,70,89,0.0,1024,67,68,76,62,7.0,5,14,0.000978,0.079365,0.072289,0.0
4,7,81,0.0,1023,71,71,79,62,10.9,6,17,-0.000977,0.044118,-0.089888,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,74,75,0.1,1010,51,54,59,49,6.6,3,10,0.002979,-0.035714,-0.050633,0.0
183,77,91,0.1,1016,48,50,52,47,6.6,3,5,0.005941,-0.074074,0.213333,0.0
184,66,77,0.1,1008,49,52,55,45,7.8,3,10,-0.007874,0.040000,-0.153846,0.0
185,40,70,0.0,1009,38,44,50,38,7.8,2,12,0.000992,-0.153846,-0.090909,0.0


# Machine Learning Pre-Processing of Data

In [16]:
hist_ml_df=history_df.drop(columns =["Dates"])
hist_ml_df

,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange,Migraine
1,9,63,0.0,1028,49,51,59,44,11.0,4,21.0,0.001949,0.040816,-0.086957,1.0
2,40,83,0.0,1026,55,56,65,49,11.0,3,22.0,-0.001946,0.098039,0.317460,0.0
3,45,83,0.0,1023,61,63,72,54,10.1,4,30.0,-0.002924,0.125000,0.000000,0.0
4,70,89,0.0,1024,67,68,76,62,7.0,5,40.0,0.000978,0.079365,0.072289,0.0
5,7,81,0.0,1023,71,71,79,62,10.9,6,46.0,-0.000977,0.044118,-0.089888,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,74,75,0.1,1010,51,54,59,49,6.6,3,0.0,0.002979,-0.035714,-0.050633,0.0
184,77,91,0.1,1016,48,50,52,47,6.6,3,0.0,0.005941,-0.074074,0.213333,0.0
185,66,77,0.1,1008,49,52,55,45,7.8,3,0.0,-0.007874,0.040000,-0.153846,0.0
186,40,70,0.0,1009,38,44,50,38,7.8,2,0.0,0.000992,-0.153846,-0.090909,0.0


In [17]:
# Assign X (data) and y (target)
X = hist_ml_df.drop("Migraine", axis=1)
y = hist_ml_df["Migraine"]
print(X.shape, y.shape)

(187, 14) (187,)


In [18]:
#Split our data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)

# Standard Scaler

In [19]:
#Fit on the training data
X_scaler = StandardScaler().fit(X_train)
X_scaler.get_params()

{'copy': True, 'with_mean': True, 'with_std': True}

In [20]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_test_scaled

array([[-1.40608327e+00, -1.38068669e+00, -4.14964516e-01,
         1.52988174e+00,  8.53054808e-01,  7.74721457e-01,
         8.90483854e-01,  5.69700738e-01,  1.47697912e+00,
         1.59452684e+00, -2.03172325e-01, -5.54739041e-01,
        -6.40763164e-02,  6.02922022e-01],
       [ 1.43732957e+00,  4.89658762e-01,  5.59784347e-01,
         5.18901224e-01, -1.20477587e+00, -1.13128782e+00,
        -1.35054742e+00, -7.42418470e-01, -1.13841492e+00,
        -1.07360572e+00, -2.03172325e-01, -5.58931061e-01,
         1.00884652e+00,  1.82015421e+00],
       [ 1.06237403e+00,  1.74255787e-03, -4.14964516e-01,
        -1.21420823e+00,  2.65103187e-01,  3.20909725e-01,
         1.43473431e-01,  6.63423538e-01, -5.34862451e-01,
        -1.84228201e-01, -2.03172325e-01,  3.75424068e-01,
         8.83274953e-02,  3.46409793e-01],
       [-3.12462950e-01, -3.23534911e-01, -4.14964516e-01,
        -2.03227716e-01, -6.90318199e-01, -8.59000779e-01,
        -8.52540468e-01, -5.54972869e-01,  7.

In [21]:
y_train.value_counts()

0.0    72
1.0    68
Name: Migraine, dtype: int64

In [22]:
y_test.value_counts()

0.0    28
1.0    19
Name: Migraine, dtype: int64

In [23]:
model = SVC(kernel="linear")
model

SVC(kernel='linear')

# Predictions Before Grid Search

In [24]:
model.fit(X_train, y_train)

SVC(kernel='linear')

In [25]:
print('Test Acc: %.3f' % model.score(X_test_scaled, y_test))

Test Acc: 0.596


In [26]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=["0","1"]))

              precision    recall  f1-score   support

           0       0.67      0.71      0.69        28
           1       0.53      0.47      0.50        19

    accuracy                           0.62        47
   macro avg       0.60      0.59      0.59        47
weighted avg       0.61      0.62      0.61        47



In [29]:
predictions

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

# Predictions Using Grid Search

In [30]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [31]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.429, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.607, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.714, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.357, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.464, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.429, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ...........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] .................... C=5, gamma=0.005, score=0.571, total=   0.0s
[CV] C=5, gamma=0.005 ................................................
[CV] .................... C=5, gamma=0.005, score=0.571, total=   0.0s
[CV] C=5, gamma=0.005 ................................................
[CV] .................... C=5, gamma=0.005, score=0.357, total=   0.0s
[CV] C=5, gamma=0.005 ................................................
[CV] .................... C=5, gamma=0.005, score=0.464, total=   0.0s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .................. C=10, gamma=0.0001, score=0.429, total=   0.0s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .................. C=10, gamma=0.0001, score=0.536, total=   0.0s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .................. C=10, gamma=0.0001, score=0.571, total=   0.0s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    1.3s finished


GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10, 50],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             verbose=3)

In [32]:
# List the best parameters for this dataset
print(grid.best_params_)

{'C': 1, 'gamma': 0.0001}


In [33]:
# List the best score
print(grid.best_score_)

0.5142857142857143


In [34]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [69]:
print(grid.score(X_test_scaled, y_test))

0.574468085106383


In [36]:
predictions

array([1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [37]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                             target_names=["0","1"]))

              precision    recall  f1-score   support

           0       0.62      0.75      0.68        28
           1       0.46      0.32      0.37        19

    accuracy                           0.57        47
   macro avg       0.54      0.53      0.53        47
weighted avg       0.55      0.57      0.56        47



In [38]:
# history_df.to_csv("../../Desktop/test.csv")

# Run Model on Forecast Data to Formulate Predictions

In [39]:
X_new_scaled =X_scaler.transform(new_migraine_df)
model.predict(X_new_scaled)

array([0., 0., 0., 0., 0., 0., 0.])

In [42]:
forecast_predictions = grid.predict(X_new_scaled)
forecast_predictions

array([0., 0., 0., 0., 0., 0., 0.])

In [68]:
lists = forecast_predictions.tolist()
json_str = json.dumps(lists)
json_str

'[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]'